# **Computer Vision Assignment 5**

In [ ]:
! pip install aicrowd-cli

In [7]:
API_KEY = "375767f7b488df578d5c7e7ef9545f2d" #Please enter your API Key from [https://www.aicrowd.com/participants/me]
! aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [8]:
! aicrowd dataset download --challenge chunin-exams-food-track-cv-2021

train_images.zip: 100% 754M/754M [00:19<00:00, 39.5MB/s]
test_images.zip: 100% 33.9M/33.9M [00:01<00:00, 27.2MB/s]
train.csv: 100% 253k/253k [00:00<00:00, 1.03MB/s]
test.csv: 100% 7.27k/7.27k [00:00<00:00, 917kB/s]


In [5]:
! wget https://gitlab.aicrowd.com/aicrowd/practice-challenges/aicrowd_FOODC_challenge/-/raw/master/dataset_info.txt

--2021-04-18 11:19:51--  https://gitlab.aicrowd.com/aicrowd/practice-challenges/aicrowd_FOODC_challenge/-/raw/master/dataset_info.txt
Resolving gitlab.aicrowd.com (gitlab.aicrowd.com)... 18.194.250.154, 18.195.114.113
Connecting to gitlab.aicrowd.com (gitlab.aicrowd.com)|18.194.250.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3451 (3.4K) [text/plain]
Saving to: ‘dataset_info.txt’

dataset_info.txt    100%[===================>]   3.37K  --.-KB/s    in 0s      

2021-04-18 11:19:52 (526 MB/s) - ‘dataset_info.txt’ saved [3451/3451]



In [151]:
# library imports
import os
import tqdm
import random
import time
import shutil
import cv2
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt

In [4]:
! rm -r data

In [5]:
! unzip train_images.zip
! unzip test_images.zip

! mkdir data
! mkdir data/train
! mkdir data/val

In [6]:
! ls -1 test_images | wc -l
! cat test.csv | wc -l

! ls -1 train_images | wc -l
! cat train.csv | wc -l

484
485
9323
9324


Creating label array

In [7]:
with open('dataset_info.txt', 'r') as f:
  labels = f.read()

labels = labels.split('\n')[1:]
labels = [ i.strip().split('. ')[-1] for i in labels ]
labels[:5]

['BG', 'water', 'pizza-margherita-baked', 'broccoli', 'salad-leaf-salad-green']

Making label -> index and index -> label mapping

In [8]:
lbl2idx = {}
for i in range(len(labels)):
  lbl2idx[labels[i]] = i

idx2lbl = {}
for i in range(len(labels)):
  idx2lbl[i] = labels[i]

In [9]:
for i in labels:
  os.mkdir('data/train/' + i)
  os.mkdir('data/val/' + i)

Splitting data into train and val

In [10]:
classwise_alltrain_images = {}
classwise_train_images = {}
classwise_val_images = {}

with open('train.csv', 'r') as f:
  trainData = f.read()

trainData = trainData.split('\n')[1:-1]
trainData = [ i.split(',') for i in trainData ]
for i in trainData:
  try:
    classwise_alltrain_images[i[1]]
    classwise_alltrain_images[i[1]].append(i[0])
  except:
    classwise_alltrain_images[i[1]] = []

In [11]:
for i in classwise_alltrain_images.keys():
  sizee = int(len(classwise_alltrain_images[i]) * 0.7)
  for j in range(sizee):
    classwise_alltrain_images[i].pop()

In [12]:
for i in classwise_alltrain_images.keys():
  trainSamples = int(len(classwise_alltrain_images[i]) * 0.8)
  trainSamples = random.sample(range(0, len(classwise_alltrain_images[i])), trainSamples)
  testSamples = [ j for j in range(len(classwise_alltrain_images[i])) if j not in trainSamples ]
  classwise_train_images[i] = [classwise_alltrain_images[i][j] for j in trainSamples]
  classwise_val_images[i] = [classwise_alltrain_images[i][j] for j in testSamples]

In [13]:
for i in classwise_train_images.keys():
  for j in classwise_train_images[i]:
    shutil.copyfile('train_images/' + j, 'data/train/' + i + '/' + j)

In [14]:
for i in classwise_val_images.keys():
  for j in classwise_val_images[i]:
    shutil.copyfile('train_images/' + j, 'data/val/' + i + '/' + j)

Making list of all training and validation samples

In [15]:
train_images_list = []
train_labels_list = []
for i in classwise_train_images.keys():
  for j in classwise_train_images[i]:
    train_images_list.append('data/train/' + i + '/' + j)
    train_labels_list.append(lbl2idx[i])

val_images_list = []
val_labels_list = []
for i in classwise_val_images.keys():
  for j in classwise_val_images[i]:
    val_images_list.append('data/val/' + i + '/' + j)
    val_labels_list.append(lbl2idx[i])

Custom Food Dataset Dataset

In [209]:
class FoodData(Dataset):
  def __init__(self, list_images, list_labels):
    self.list_images = list_images
    self.list_labels = list_labels

  def __len__(self):
    return len(self.list_labels)

  def __getitem__(self, index):
    img = cv2.imread(self.list_images[index])
    img = cv2.resize(img, (224, 224)).astype(np.float64) / 255
    lbl = np.zeros(len(labels)).astype(np.float)
    lbl[self.list_labels[index]] = 1
    return {'feature': img, 'label': lbl}

Creating Dataset object

In [210]:
train_dl = FoodData(train_images_list, train_labels_list)
val_dl = FoodData(val_images_list, val_labels_list)

In [211]:
train_dl.__getitem__(0)['feature'].shape

(65536,)

Dataloader

In [212]:
train_dataloader = torch.utils.data.DataLoader(train_dl, shuffle = False, batch_size = 128, num_workers = 2)
val_dataloader = torch.utils.data.DataLoader(val_dl, shuffle = True, batch_size = 128, num_workers = 2)

Model

In [213]:
class FoodClassifier(nn.Module):
  def __init__(self):
    super(FoodClassifier, self).__init__()
    self.layer1 = models.densenet161(pretrained=True)
    self.layer1.fc = nn.Sequential(
                  nn.Linear(1024, 256, bias=True), 
                  nn.ReLU(), 
                  nn.Dropout(0.4),
                  nn.Linear(256, len(labels)),                   
                  nn.LogSoftmax(dim=1)
              )

  def forward(self, x):
    return self.layer1(x)

Training code

In [214]:
def check_cuda():
    _cuda = False
    if torch.cuda.is_available():
        _cuda = True
    return _cuda

In [215]:
def train_model(model, train_loader, val_loader):
    n_epochs = 10
    bestModel = model
    bestLoss = 1000000
    
    for epoch in range(n_epochs):
        train_loss = 0.0
        model.train()
        for data_tmp in tqdm.tqdm(train_loader):
            # move tensors to GPU
            data, target = data_tmp['feature'].cuda().float(), data_tmp['label'].cuda().float()
            optimizer.zero_grad()
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
        # calculate average losses
        train_loss = train_loss/len(train_loader.dataset)
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
        lossVal = val_model(model, val_dataloader)
        train_losses.append(train_loss)
        if lossVal < bestLoss:
          bestLoss = lossVal
          bestModel = model
    return model

In [216]:
def val_model(model, test_loader):
    losss = 0.0
    correct = 0
    total = 0

    model.eval()
    for data_tmp in tqdm.tqdm(test_loader):
        # move tensors to GPU
        data, target = data_tmp['feature'].cuda().float(), data_tmp['label'].cuda().float()
        output = model(data)
        loss = criterion(output, target)
        losss += loss.item()*data.size(0)
        _, pred = torch.max(output, 1)
        _, gt = torch.max(target, 1)
        correct += (pred == gt).sum().cpu().detach().numpy()
        total += len(gt)
    val_losses.append(losss)
    print('\nValidation loss =',losss)
    print('Validation accuracy =', (correct / total) * 100, '%')
    return losss

In [ ]:
train_model(model, train_loader, val_loader)

In [ ]:
val_model(model, val_loader)